# 기온, 기압, 습도 날짜별 차 구하기

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
data = pd.read_csv('../2013-2018년_가스공급량_기온_습도_기압01.csv')

In [3]:
data.head()

,year,month,day,weekday,시간,구분,공급량,기온,습도,기압
0,2013,1,1,1,1,0,2497.129,-8.5,57.0,1010.0
1,2013,1,1,1,1,1,2169.093,-8.5,57.0,1010.0
2,2013,1,1,1,1,2,226.178,-8.5,57.0,1010.0
3,2013,1,1,1,1,3,1434.516,-8.5,57.0,1010.0
4,2013,1,1,1,1,4,3272.837,-8.5,57.0,1010.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   year     368088 non-null  int64  
 1   month    368088 non-null  int64  
 2   day      368088 non-null  int64  
 3   weekday  368088 non-null  int64  
 4   시간       368088 non-null  int64  
 5   구분       368088 non-null  int64  
 6   공급량      368088 non-null  float64
 7   기온       368088 non-null  float64
 8   습도       368088 non-null  float64
 9   기압       368088 non-null  float64
dtypes: float64(4), int64(6)
memory usage: 28.1 MB


## 데이터 구분별로 정렬

In [5]:
data = data.sort_values(['구분', 'year', 'month', 'day', 'weekday', '시간']).reset_index(drop=True)
data

,year,month,day,weekday,시간,구분,공급량,기온,습도,기압
0,2013,1,1,1,1,0,2497.129,-8.5,57.0,1010.0
1,2013,1,1,1,2,0,2363.265,-8.4,60.0,1009.4
2,2013,1,1,1,3,0,2258.505,-8.1,58.0,1009.2
3,2013,1,1,1,4,0,2243.969,-8.2,58.0,1008.2
4,2013,1,1,1,5,0,2344.105,-8.2,61.0,1007.3
...,...,...,...,...,...,...,...,...,...,...
368083,2018,12,31,0,20,6,681.033,-3.7,39.0,1024.9
368084,2018,12,31,0,21,6,669.961,-4.6,44.0,1024.8
368085,2018,12,31,0,22,6,657.941,-5.4,46.0,1024.4
368086,2018,12,31,0,23,6,610.953,-5.2,47.0,1024.6


## 전날 기준 차 구하기

In [6]:
divisions = data['구분'].unique()
minus_data = pd.DataFrame()
for division in tqdm(divisions):
    # 구분별로 데이터 불러오기
    temp_data = data[data['구분'] == division].reset_index(drop=True)
    temp_data = temp_data[['공급량', '기온', '습도', '기압']]
    
    # 전날 기준으로 비율 구하기(첫 째날은 비교 대상 없으므로 0으로 통일)
    temp_minus = temp_data.iloc[0,:] -  temp_data.iloc[0,:]
    for i in range(1, len(temp_data)):
        present = temp_data.iloc[i,:]
        before = temp_data.iloc[i-1,:]
        temp = present - before
        temp_minus = pd.concat([temp_minus, temp],axis=1, ignore_index=True)
    # 행열 전환
    temp_minus = temp_minus.T
    
    # 구분별 차 데이터 하나로 합치기
    minus_data = pd.concat([minus_data, temp_minus], ignore_index=True)

100%|██████████| 7/7 [16:02<00:00, 137.56s/it]


In [13]:
minus_data.head()

,공급량차,기온차,습도차,기압차
0,0.000,0.0,0.0,0.0
1,-133.864,0.1,3.0,-0.6
2,-104.760,0.3,-2.0,-0.2
3,-14.536,-0.1,0.0,-1.0
4,100.136,0.0,3.0,-0.9


In [14]:
minus_data.columns = ['공급량차', '기온차', '습도차', '기압차']
minus_data.head()

,공급량차,기온차,습도차,기압차
0,0.000,0.0,0.0,0.0
1,-133.864,0.1,3.0,-0.6
2,-104.760,0.3,-2.0,-0.2
3,-14.536,-0.1,0.0,-1.0
4,100.136,0.0,3.0,-0.9


In [15]:
final_minus_data = pd.concat([data, minus_data], axis=1)
final_minus_data.head()

,year,month,day,weekday,시간,구분,공급량,기온,습도,기압,공급량차,기온차,습도차,기압차
0,2013,1,1,1,1,0,2497.129,-8.5,57.0,1010.0,0.000,0.0,0.0,0.0
1,2013,1,1,1,2,0,2363.265,-8.4,60.0,1009.4,-133.864,0.1,3.0,-0.6
2,2013,1,1,1,3,0,2258.505,-8.1,58.0,1009.2,-104.760,0.3,-2.0,-0.2
3,2013,1,1,1,4,0,2243.969,-8.2,58.0,1008.2,-14.536,-0.1,0.0,-1.0
4,2013,1,1,1,5,0,2344.105,-8.2,61.0,1007.3,100.136,0.0,3.0,-0.9


In [16]:
final_minus_data.tail()

,year,month,day,weekday,시간,구분,공급량,기온,습도,기압,공급량차,기온차,습도차,기압차
368083,2018,12,31,0,20,6,681.033,-3.7,39.0,1024.9,2.058,-0.6,-4.0,-0.6
368084,2018,12,31,0,21,6,669.961,-4.6,44.0,1024.8,-11.072,-0.9,5.0,-0.1
368085,2018,12,31,0,22,6,657.941,-5.4,46.0,1024.4,-12.020,-0.8,2.0,-0.4
368086,2018,12,31,0,23,6,610.953,-5.2,47.0,1024.6,-46.988,0.2,1.0,0.2
368087,2018,12,31,0,24,6,560.896,-5.5,54.0,1024.1,-50.057,-0.3,7.0,-0.5


In [17]:
final_minus_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   year     368088 non-null  int64  
 1   month    368088 non-null  int64  
 2   day      368088 non-null  int64  
 3   weekday  368088 non-null  int64  
 4   시간       368088 non-null  int64  
 5   구분       368088 non-null  int64  
 6   공급량      368088 non-null  float64
 7   기온       368088 non-null  float64
 8   습도       368088 non-null  float64
 9   기압       368088 non-null  float64
 10  공급량차     368088 non-null  float64
 11  기온차      368088 non-null  float64
 12  습도차      368088 non-null  float64
 13  기압차      368088 non-null  float64
dtypes: float64(8), int64(6)
memory usage: 39.3 MB


In [18]:
final_minus_data.to_csv('2013-2018년_가스공급량_기온_습도_기압_차.csv', index=False)